# Introduction

<div class="alert alert-info">

Summary

The model parameterisations refer to the set of parameters that are varied and estimated in a retrieval. Given that the computational time of retrievals largely depend on the number of fitted parameters, the input information in the atmospheric classes is often parameterised to reduce this number. ArchNEMESIS includes a large set of model parameterisations heritage from the original NEMESIS code and which might be useful for many applications. Nevertheless, new parameterisations can also be easily introduced to develop new methodologies for the analysis of planetary spectra.

</div>

The information about the model parameterisations is stored in the *Variables* class. Given the different forms parameterisations may take, this class needs to be versatile to store all the relevant information about the model parameters and the indications of what information must be changed in the forward model based on a particular parameterisation.

The type of parameterisation is indicated by the VARIDENT parameter of the *Variables* class. For each parameterisation included in the model, VARIDENT is a three-element integer array. The logic behind this array is as follows:

- Let's call the three elements of the array I$_1$, I$_2$, I$_3$.

- I$_3$ indicates the ID for the particular parameterisation, following the identifiers listed in the next section.

- If the selected model ID does not correspond to an atmospheric model, then I$_1$ and I$_2$ do not provide any useful information, and they can get arbitrary numbers. However, if the model ID corresponds to an atmospheric parameterisation, then I$_1$ and I$_2$ indicate the atmospheric parameter that must be modified. In particular:

    - If I$_1$ = 0, then the model parameterisation applies to the temperature profile.
    - If I$_1$ > 0, then the model parameterisation applies to the volume mixing ratio of a particular gas, with I$_1$ and I$_2$ representing the gas ID and isotope ID (e.g., I$_1$=1, I$_2$=0 for H$_2$O;  I$_1$=1, I$_2$=4 for HDO).
    - If I$_1$ < 0, then we can have:
        - If 1 $\leq$ |I$_1$| $\leq$ NDUST, then the model parameterisation applies to the I$_1$ aerosol population.
        - If |I$_1$| = NDUST + 1, then the model parameterisation applies to the para-H$_2$ fraction profile.
        - If |I$_1$| = NDUST + 1, then the model parameterisation applies to the fractional cloud cover profile.


<div class="alert alert-warning" >

Examples of VARIDENT

Let's assume we have an atmosphere with 3 aerosol populations. We include 5 model parameterisations, defined by:

- I$_1$=2, I$_2$=0, I$_3$=0: Continuous vertical profile of CO$_2$ volume mixing ratio.
- I$_1$=-2, I$_2$=0, I$_3$=2: Scaling factor for abundance of the second aerosol population in the atmosphere.
- I$_1$=-4, I$_2$=0, I$_3$=0: Scaling factor for para-H2 fraction profile.
- I$_1$=-1, I$_2$=0, I$_3$=32: Abundance of first aerosol population represented by base pressure, optical depth and fractional scale height.
- I$_1$=0, I$_2$=0, I$_3$=231: Multiplication of spectrum by polynomial function

</div>

# Available models

The model parameterisations have the purpose of modifying the information in the reference classes according to a smaller number of parameters. This is especially important for retrievals, when these parameters are modified in each iteration to find the best fit between the modelled and measured spectra. Each of the model parameterisations is defined by some Variable IDs and Variable parameters, which are defined in the *.apr* file. This file can be read using the *read_apr* function on the Variables class. 

At this stage, only a number of models have been implemented in archNEMESIS with respect to NEMESIS. Detailed information about each of the model parameterisations is included in the Examples. In the following, a brief summary describing each of these models is included:


| Model ID | No. of parameters | No. of extra parameters | Description |
| --- | --- | --- | --- |
| 49 | NP | 0 | Continuous atmospheric vertical profile in linear scale (i.e., allowing negative abundances) |
| 50 | NP | 0 | Continuous profile of scaling factors in linear scale |
| 2310 | NGEOM $\times$ (NDEGREE+1) $\times$ NWINDOWS | 3 | Same as model 231 but applying it to different spectral windows |

# Custom models

Users may want to define their own custom parameterisations ande create new retrieval schemes for their specific purposes. Implementing new parameterisations in archNEMESIS is relatively straightforward, just needing to follow the steps described below. We recommend the users to check how other parameterisations are also included in the relevant functions listed below. 

1. Select one model parameterisation ID number, considering the numbers already taken by others. It is also advisable to check the numbers already taken by parameterisations implemented in the [NEMESIS code](https://nemesiscode.github.io/manuals.html) - we want to respect the ID numbers from the original NEMESIS code to ensure backward compatibility between the two versions.
 
2. **Implement changes in the *Variables* class**. Open the file for the class (*Variables_0.py*), and perform the following changes:
    - In the *read_apr()* method, we need to describe the way the parameterisation is read from the input .apr file. In particular, we need to define the following variables from the file.
       - *Variables*.VARIDENT : This field represents the Variable IDs, unique for this model parameterisation.
       - *Variables*.VARPARAM : This field represents any extra parameters the code needs to read to properly represent the parameterisation, but that will not be retrieved (i.e., they are not included in the state vector).
       - *Variables*.XA : This field represents the a priori state vector and must be filled accordingly based on the the parameters stored in the .apr file.
       - *Variables*.SA : This field represents the a priori covariance matrix and must be filled accordingly based on the parameter uncertainties stored in the .apr file.
       - *Variables*.LX : This field tells whether a particular element of the state vector is carried in log-scale (1) or not (0).
       - *Variables*.NUM : This field tells whether the jacobian matrix for this particular element of the state vector can be computed analytically (0) or numerically (1).

    - In the *calc_NXVAR()* method, we need to indicate the number of parameters in the state vector associated with this particular model parameterisation.
   
3. **Define the model parameterisation.** So far we have only indicated how we want to read the model parameters, so we need to implement the model itself. All model parametersiations in archNEMESIS are listed in the *Models.py* file. The idea of these models is for them to produce a change in either one of the parameters of the reference classes (e.g., temperature field in the *Atmosphere* class), or that they produce a change in the spectrum itself (e.g., multiplication of the spectrum by a dilution factor). Therefore, the inputs of our function will be the model parameters + reference class, or model parameters + spectrum. The function must apply the model and return the updated reference class or spectrum. Some of the model implementations also include the partial derivatives of the state vector, with respect to the parameter in the model. If this is the case (i.e., *Variables*.NUM=0), the model implementation must also include this.
    
4. **Implement the new paramerterisation into the forward model**. The model parameterisations are integrated with the radiative transfer calculations in the *ForwardModel* class. There are two main functions were the parameterisations are implemented:
   - In the case that our parameterisation affects the input information (i.e., the reference classes), we need to update the *subprofretg()* method of the class to modify the reference classes based on the parameters in the state vector, as well as the gradients in the case that these are calculated analytically.
   - In the case that our parameterisation affects the output spectrum, we need to updated the *subspecret()* method of the class to modify the spectrum based on the parameters in the state vector, as well as the gradients in the case that these are calculated analytically.
